# Facilities KPI Scorecard
This notebook serves as a repository of public-facing KPIs for the Department of General Services' Facilities Division. The purpose of this notebook is to have, in a single public place, the entire pipeline for calculating the division's three KPIs that are based in Archibus. Those KPIs are:

1. % of Corrective Maintenance Work Requests Completed On Time
2. % of Preventive Maintenance Work Requests Completed On Time
3. Preventive Maintenace to Corrective Maintenance Ratio

Part of the intention of this notebook is to build _transparency_ and _reproducibility_ by capturing the entire end-to-end process used to calculate these metrics in one place.

## Setup

### Import packages

In [1]:
# workhorse modules
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import re
from pathlib import Path
import seaborn as sns
import datadotworld as dw
import matplotlib.pyplot as plt

# local utility functions
from utils import (
    add_cm_benchmarks,
    add_fiscal_year,
    set_pd_params,
    tidy_up_df,
    cast_dtypes,
    glue_date_time,
    compute_days_to_completion,
    consolidate_prob_types,
    compute_pm_cm,
    compute_kpi_table,
)

from vis_utils import set_plot_params, pointplot_with_barplot

### Set pandas options
This makes Pandas print all rows and columns to the output when requested.

In [2]:
set_pd_params()
set_plot_params()
pd.options.mode.chained_assignment = None  # default='warn'

### Import the work order data from Data.World
This data is a copy of Archibus's `wrhwr` table stored at DGS's account on Data.World. To see the exact query used to generate the input data, see `/sql/input_for_FMD_KPIs.sql`.

In [3]:
kpis_raw = dw.query(
    dataset_key="dgsbpio/auditfinding3", query="select * from wrhwr_01072021"
).dataframe

In [4]:
print(f"The work orders dataframe has {kpis_raw.shape[0]:,} rows.")

kpis_raw.sample(3, random_state=444)[["wr_id", "prob_type", "date_requested"]]

The work orders dataframe has 105,391 rows.


,wr_id,prob_type,date_requested
12670,12855,_DELIVERY,2014-07-09
77427,107443,OTHER,2018-06-27
39319,63972,DOOR,2016-03-29


### Import the users table
The only reason why we need this second table is to figure out which work requests of type "OTHER" were requested by FMD staff and which were requested by a Service Request Liaison. 

OTHERs requested internally will be given the primary problem type "OTHER-INTERNAL", while OTHERs requested by SRLs will be given the primary problem type "OTHER-EXTERNAL".

In [5]:
users = dw.query(
    dataset_key="dgsbpio/auditfinding3", query="select * from archibus_user_roles"
).dataframe

users = users.applymap(lambda x: x.strip() if isinstance(x, str) else x)

## Data cleaning
For the purposes of this project, DGS is keeping the cleaning stage very simple. We're not attempting to remove duplicates or outliers, both of which involve relatively complex operations. 

### Basic cleaning
- removes white spaces in strings to facilitate matching, 
- drops rows with no problem type, or rows created by a test,
- renames a few columns

In [6]:
# make sure each column is of the correct data type
wr_tidy = cast_dtypes(kpis_raw)
# basic cleaning
wr_tidy = tidy_up_df(wr_tidy)

print(f"The tidied work orders dataframe has {len(wr_tidy):,} rows.")
print(f"By tidying the data, we have removed {len(kpis_raw) - len(wr_tidy):,} rows.")

The tidied work orders dataframe has 105,266 rows.
By tidying the data, we have removed 125 rows.


### Drop canceled work orders
The data comes to us with many canceled work orders. Those shouldn't count against us as not having been completed on time. So we drop them here. 

In [7]:
# drop rows that were canceled
cond_valid = wr_tidy["status"] != "Can"

wr_valid = wr_tidy[cond_valid]

print(f"By dropping canceled work ordered, we have removed {len(wr_tidy) - len(wr_valid):,} rows.")

By dropping canceled work ordered, we have removed 3,274 rows.


### Combine date and time columns to get timestamps
This takes the date from a date column and the time from a time column and combines them into a single timestamp.

This transformation allows us to know the time to completion with greater precision. 

In [ ]:
# glue the date and time for request
wr_dt = glue_date_time(wr_valid, "date_requested", "time_requested", "requested_dt")

# glue the date and time for completion
wr_dt = glue_date_time(wr_dt, "date_completed", "time_completed", "completed_dt")

# convert "date closed" to a valid datetime (this column has no time information)
wr_dt["date_closed"] = wr_dt["date_closed"].astype("datetime64")

### Examine the cleaned data
The display below gives us a sense of what some key columns in the data now look like. 

In [ ]:
wr_dt[
    [
        "wr_id",
        "problem_type",
        "requested_dt",
        "completed_dt",
        "date_closed",
        "status",
    ]
].sample(3, random_state=451)

## Data preparation

### Merge work requests and user data
Now we bring the data from the users table into the work request table. This lets us figure out the rose of the user who requested each work request. 

In [ ]:
wr_joined = wr_dt.merge(
    users[["user_name", "role_name"]],
    how="left",
    left_on="requestor",
    right_on="user_name",
)

### Add days to completion
Next, we compute the duration of time between the request and completion, for each job. We store that number in a column called "days_to_completion". 

In [ ]:
wr_durations = compute_days_to_completion(wr_joined)

Let's check out three rows of the data, now that we've added some columns. Notice that `days_to_completion` is empty if the work request hasn't been marked completed yet. 

In [ ]:
wr_durations[
    [
        "wr_id",
        "role_name",
        "problem_type",
        "requested_dt",
        "completed_dt",
        "date_closed",
        "days_to_completion",
        "status",
    ]
].sample(3, random_state=446)

### Add fiscal year
There are different ways to assign a fiscal year to a work request. Currently, we're doing it by the fiscal year in which the work request was closed. Other options would include the fiscal year of the request. 


In [ ]:
wr_fy = add_fiscal_year(wr_durations, assign_fy_on="closure")

After assigning the fiscal year, we can drop work requests from FY15 and older. 

In [ ]:
cond_fy = wr_fy["fiscal_year"].isin(range(2016, 2022))
wr_fy = wr_fy[cond_fy]

## KPI: % PMs Completed On Time 
The goal here is to filter the data down to preventive maintenance only, and then show how many are completed on or before the benchmark (21 days).

### Filter to valid PM only, and for relevant fiscal years only
After noticing that many PMs have the status "Can" — meaning they were canceled — we fil

In [ ]:
# this defines which problem types are considered PMs
pm_list = [
    "BUILDING INTERIOR INSPECTION",
    "BUILDING PM",
    "HVAC|PM",
    "INSEPCTION",
    "PREVENTIVE MAINT",
]

# filter data to PM types only
cond_pm = wr_fy["problem_type"].isin(pm_list)

# apply filter conditions
wr_pm = wr_fy[cond_pm]

print(f"The filtered PMs dataframe has {wr_pm.shape[0]:,} rows.")

#### Compute the benchmark and add `is_on_time` column
Here we tell Python that PMs are on-time if completed in 21 days. 

Then the function `compute_is_on_time` compares the column `days_to_completion` to the benchmark and writes down whether the work request was completed on time. 

In [ ]:
def compute_is_on_time(row):
    row["is_on_time"] = row["days_to_completion"] <= row["benchmark"]
    return row


# assign benchmark
wr_pm["benchmark"] = 21

# add "is_on_time" column with performance data
pms_on_time = wr_pm.apply(compute_is_on_time, axis=1)

In [ ]:
pms_on_time[
    [
        "wr_id",
        "problem_type",
        "requested_dt",
        "completed_dt",
        "days_to_completion",
        "is_on_time",
        "status", 
    ]
].sample(3, random_state=446)

#### Group by fiscal year and get % on time
Now that we've stored all this information, we can group by the fiscal year to get each year's KPI, together with a count of how many PMs 

In [ ]:
pm_compliance = compute_kpi_table(pms_on_time, "percent_PMs_on_time", "total_PMs")
pm_compliance

In [ ]:
pointplot_with_barplot(
    pm_compliance,
    x="fiscal_year",
    point_y="percent_PMs_on_time",
    bar_y="total_PMs",
    ymax_bar=2_500,
    ylabel_point="Percent PMs On Time",
    ylabel_bar="Total PMs",
    title="Percent of PMs On Time By Fiscal Year",
)

## KPI: PM:CM Ratio

The two lists below contain the exact same problem types mentioned in last year's scorecard. So we would expect to be able to replicate last year's results closely.

In [ ]:
CM_list = [
    "BOILER",
    "CHILLERS",
    "COOLING TOWERS",
    "HVAC",
    "HVAC INFRASTRUCTURE",
    "HVAC|REPAIR",
]

PM_list = [
    "HVAC|PM",
    "PREVENTIVE MAINT",
]

### Filter to HVAC rows only

In [ ]:
cond_cm = wr_fy["problem_type"].isin(CM_list)
cond_pm = wr_fy["problem_type"].isin(PM_list)

wr_HVAC = wr_fy[cond_cm | cond_pm]
wr_HVAC["is_pm"] = wr_HVAC["problem_type"].isin(PM_list)

print(f"We've gone from {len(wr_fy):,} rows to {len(wr_HVAC):,} rows.")

### Compute all PM/CM stats by fiscal year

In [ ]:
def compute_pm_cm(df, PM_list):
    df = df.copy().sort_values("fiscal_year")
    results_df = pd.DataFrame(
        columns=[
            "fiscal_year",
            # "percent_pm",
            "pm_cm_ratio",
            "count_cm",
            "count_pm",
            "count_hvac",
        ]
    )
    for fiscal_year in df["fiscal_year"].unique():
        results_dict = {}
        df_fy = df[df["fiscal_year"] == fiscal_year]
        cond_pm = df_fy["is_pm"] == True
        count_pm = len(df_fy[cond_pm])
        count_hvac = len(df_fy)
        count_cm = count_hvac - count_pm
        results_dict["fiscal_year"] = fiscal_year
        results_dict["pm_cm_ratio"] = (count_pm / count_cm) * 100
        results_dict["count_pm"] = count_pm
        results_dict["count_cm"] = count_cm
        results_dict["count_hvac"] = count_hvac
        results_df = results_df.append(results_dict, ignore_index=True)
    results_df[["fiscal_year", "count_cm", "count_pm", "count_hvac"]] = results_df[
        ["fiscal_year", "count_cm", "count_pm", "count_hvac"]
    ].astype(int)
    return results_df.round(2)

pm_cm_results = compute_pm_cm(wr_HVAC, PM_list)

In [ ]:
pm_cm_results

In [ ]:
# plt.figure(figsize=(14,6)) 
# count_plot_data = pd.melt(
#     pm_cm_results, id_vars=["year"], value_vars=["count_cm", "count_pm"]
# )

# ax = sns.lineplot(data=count_plot_data, y="value", x="year", hue="variable")
# ax.set(title="Volume of HVAC PM and CM Work Requests Per Fiscal Year", ylim = (0, 3000))
# sns.despine()

pointplot_with_barplot(
    pm_cm_results,
    x="fiscal_year",
    point_y="pm_cm_ratio",
    bar_y="count_hvac",
    ymax_bar=5_000,
    title="Preventive to Corrective Ratio By Fiscal Year",
)

## KPI: Percent of CM Work Requests Completed On-Time
Here are the key facts needed to understand the agency's new method for computing this KPI:

- Only CM problem types are considered, so all PM work orders are dropped.
- The work orders are first assigned a "primary" problem type, which consolidates the number of problem types
- Each of these primary problem types has a benchmark, which is then added to the work request's row
- Finally, the work order is determined to be on-time based on comparing its time to completion to its benchmark

In [ ]:
wr_cm = wr_fy.copy()

consolidated_wrs = wr_cm.apply(consolidate_prob_types, axis=1)

cond_cm = consolidated_wrs["primary"] != "PREVENTIVE"
consolidated_cms = consolidated_wrs[cond_cm]

print(
    f"Dropping some unbenchmarked small categories takes us from {len(consolidated_wrs):,} rows to {len(consolidated_cms):,} rows."
)

In [ ]:
cms_benchmarked = consolidated_cms.apply(add_cm_benchmarks, axis=1)

cms_benchmarked.sample(6, random_state=444)[
    ["problem_type", "primary", "benchmark", "days_to_completion"]
]

In [ ]:
cms_on_time = cms_benchmarked.apply(compute_is_on_time, axis=1)

In [ ]:
cm_compliance = compute_kpi_table(cms_on_time, "percent_CMs_on_time", "total_CMs")
cm_compliance

In [ ]:
pointplot_with_barplot(
    cm_compliance,
    x="fiscal_year",
    point_y="percent_CMs_on_time",
    bar_y="total_CMs",
    ymax_bar=15_000,
    ylabel_point="Percent CMs On Time",
    ylabel_bar="Total CMs",
    title="Percent of PMs On Time By Fiscal Year",
)